In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.types import StructType,StructField, StringType, IntegerType, FloatType
import pyspark.sql.functions as F

In [0]:

spark = SparkSession.builder \
    .master('local[*]') \
    .appName("Iniciando com Spark") \
    .getOrCreate()

### Lendo Csv

In [0]:
path_orders = '/FileStore/transient/olist/orders'
df_orders = spark.read.format('csv')\
.option("header", True)\
.option("sep", ",")\
.option("quote","\"")\
.option("inferSchema",True)\
.load(path_orders)
#transient\csv\olist

### Selecionando e manipulando os dados

In [0]:
df_orders.select('order_id','customer_id','order_status','order_purchase_timestamp','order_approved_at','order_delivered_carrier_date',\
                'order_delivered_customer_date','order_estimated_delivery_date').show(5,truncate=False)

+--------------------------------+--------------------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|order_id                        |customer_id                     |order_status|order_purchase_timestamp|order_approved_at  |order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|
+--------------------------------+--------------------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|e481f51cbdc54678b7cc49136f2d6af7|9ef432eb6251297304e76186b10a928d|delivered   |2017-10-02 10:56:33     |2017-10-02 11:07:15|2017-10-04 19:55:00         |2017-10-10 21:25:13          |2017-10-18 00:00:00          |
|53cdb2fc8bc7dce0b6741e2150273451|b0830fb4747a6c6d20dea0b8c802d7ef|delivered   |2018-07-24 20:41:37     |2018-07-26 03:24:27|2018-07-26 14:3

In [0]:
df_orders.printSchema()

root
 |-- order_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- order_status: string (nullable = true)
 |-- order_purchase_timestamp: timestamp (nullable = true)
 |-- order_approved_at: timestamp (nullable = true)
 |-- order_delivered_carrier_date: timestamp (nullable = true)
 |-- order_delivered_customer_date: timestamp (nullable = true)
 |-- order_estimated_delivery_date: timestamp (nullable = true)



In [0]:
df_orders.select('order_id','customer_id','order_status','order_purchase_timestamp','order_approved_at','order_delivered_carrier_date',\
                'order_delivered_customer_date','order_estimated_delivery_date').show(5,truncate=False)

+--------------------------------+--------------------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|order_id                        |customer_id                     |order_status|order_purchase_timestamp|order_approved_at  |order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|
+--------------------------------+--------------------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|e481f51cbdc54678b7cc49136f2d6af7|9ef432eb6251297304e76186b10a928d|delivered   |2017-10-02 10:56:33     |2017-10-02 11:07:15|2017-10-04 19:55:00         |2017-10-10 21:25:13          |2017-10-18 00:00:00          |
|53cdb2fc8bc7dce0b6741e2150273451|b0830fb4747a6c6d20dea0b8c802d7ef|delivered   |2018-07-24 20:41:37     |2018-07-26 03:24:27|2018-07-26 14:3

### Colunas

As colunas são unidades de manipulação de dados do Spark. 
Podemos referencias colunas de algumas formas <br>
* col('nome_coluna') <br>
* dataframe['nome_coluna'] <br>
* dataframe.nome_coluna <br>

In [0]:
from pyspark.sql.functions import col, round
(
df_orders.select('order_id', 'customer_id', 'order_status', 
F.split(F.col('order_approved_at'), '-').getItem(0).alias('aproved_year_at'),
F.split(df_orders['order_approved_at'], '-').getItem(1).alias('aproved_month_at'),
F.split(df_orders.order_approved_at, '-').getItem(2).alias('aproved_month_at')).show(5)
)

+--------------------+--------------------+------------+---------------+----------------+----------------+
|            order_id|         customer_id|order_status|aproved_year_at|aproved_month_at|aproved_month_at|
+--------------------+--------------------+------------+---------------+----------------+----------------+
|e481f51cbdc54678b...|9ef432eb625129730...|   delivered|           2017|              10|     02 11:07:15|
|53cdb2fc8bc7dce0b...|b0830fb4747a6c6d2...|   delivered|           2018|              07|     26 03:24:27|
|47770eb9100c2d0c4...|41ce2a54c0b03bf34...|   delivered|           2018|              08|     08 08:55:23|
|949d5b44dbf5de918...|f88197465ea7920ad...|   delivered|           2017|              11|     18 19:45:59|
|ad21c59c0840e6cb8...|8ab97904e6daea886...|   delivered|           2018|              02|     13 22:20:29|
+--------------------+--------------------+------------+---------------+----------------+----------------+
only showing top 5 rows



### Expressões

In [0]:
from pyspark.sql.functions import expr

(
    df_orders.select('order_id', 'customer_id', 'order_status', 
    F.expr('upper(order_status)'), )                    
    .show(5)
)

+--------------------+--------------------+------------+-------------------+
|            order_id|         customer_id|order_status|upper(order_status)|
+--------------------+--------------------+------------+-------------------+
|e481f51cbdc54678b...|9ef432eb625129730...|   delivered|          DELIVERED|
|53cdb2fc8bc7dce0b...|b0830fb4747a6c6d2...|   delivered|          DELIVERED|
|47770eb9100c2d0c4...|41ce2a54c0b03bf34...|   delivered|          DELIVERED|
|949d5b44dbf5de918...|f88197465ea7920ad...|   delivered|          DELIVERED|
|ad21c59c0840e6cb8...|8ab97904e6daea886...|   delivered|          DELIVERED|
+--------------------+--------------------+------------+-------------------+
only showing top 5 rows



In [0]:
from pyspark.sql.functions import expr
(
    df_orders.select('order_id', 'customer_id', 'order_status', 
    expr('upper(order_status)'),
    expr('substring(order_approved_at, 0,4) as year'),
    expr('substring(order_approved_at, 6,2) as month'),
    expr('substring(order_approved_at, 9,2) as day'))                    
    .show(5)
)

+--------------------+--------------------+------------+-------------------+----+-----+---+
|            order_id|         customer_id|order_status|upper(order_status)|year|month|day|
+--------------------+--------------------+------------+-------------------+----+-----+---+
|e481f51cbdc54678b...|9ef432eb625129730...|   delivered|          DELIVERED|2017|   10| 02|
|53cdb2fc8bc7dce0b...|b0830fb4747a6c6d2...|   delivered|          DELIVERED|2018|   07| 26|
|47770eb9100c2d0c4...|41ce2a54c0b03bf34...|   delivered|          DELIVERED|2018|   08| 08|
|949d5b44dbf5de918...|f88197465ea7920ad...|   delivered|          DELIVERED|2017|   11| 18|
|ad21c59c0840e6cb8...|8ab97904e6daea886...|   delivered|          DELIVERED|2018|   02| 13|
+--------------------+--------------------+------------+-------------------+----+-----+---+
only showing top 5 rows



In [0]:

cols = ['order_id', 'order_status', 'order_estimated_delivery_date']
df_orders.select(cols).show(10)

+--------------------+------------+-----------------------------+
|            order_id|order_status|order_estimated_delivery_date|
+--------------------+------------+-----------------------------+
|e481f51cbdc54678b...|   delivered|          2017-10-18 00:00:00|
|53cdb2fc8bc7dce0b...|   delivered|          2018-08-13 00:00:00|
|47770eb9100c2d0c4...|   delivered|          2018-09-04 00:00:00|
|949d5b44dbf5de918...|   delivered|          2017-12-15 00:00:00|
|ad21c59c0840e6cb8...|   delivered|          2018-02-26 00:00:00|
|a4591c265e18cb1dc...|   delivered|          2017-08-01 00:00:00|
|136cce7faa42fdb2c...|    invoiced|          2017-05-09 00:00:00|
|6514b8ad8028c9f2c...|   delivered|          2017-06-07 00:00:00|
|76c6e866289321a7c...|   delivered|          2017-03-06 00:00:00|
|e69bfb5eb88e0ed6a...|   delivered|          2017-08-23 00:00:00|
+--------------------+------------+-----------------------------+
only showing top 10 rows



In [0]:
cols = ['order_id', 'order_status', 'order_estimated_delivery_date']
df_orders.select('customer_id', *cols).show(10)

+--------------------+--------------------+------------+-----------------------------+
|         customer_id|            order_id|order_status|order_estimated_delivery_date|
+--------------------+--------------------+------------+-----------------------------+
|9ef432eb625129730...|e481f51cbdc54678b...|   delivered|          2017-10-18 00:00:00|
|b0830fb4747a6c6d2...|53cdb2fc8bc7dce0b...|   delivered|          2018-08-13 00:00:00|
|41ce2a54c0b03bf34...|47770eb9100c2d0c4...|   delivered|          2018-09-04 00:00:00|
|f88197465ea7920ad...|949d5b44dbf5de918...|   delivered|          2017-12-15 00:00:00|
|8ab97904e6daea886...|ad21c59c0840e6cb8...|   delivered|          2018-02-26 00:00:00|
|503740e9ca751ccdd...|a4591c265e18cb1dc...|   delivered|          2017-08-01 00:00:00|
|ed0271e0b7da060a3...|136cce7faa42fdb2c...|    invoiced|          2017-05-09 00:00:00|
|9bdf08b4b3b52b552...|6514b8ad8028c9f2c...|   delivered|          2017-06-07 00:00:00|
|f54a9f0e6b351c431...|76c6e866289321a7c...|

Observações:
* Podemos realizar operações sobre colunas selecionadas. 
* O DataFrame resultante resultante das operações vai obedeçer a order das colunas em que ele foi criado.

In [0]:
df_orders.show(2,truncate=False)

+--------------------------------+--------------------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|order_id                        |customer_id                     |order_status|order_purchase_timestamp|order_approved_at  |order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|
+--------------------------------+--------------------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|e481f51cbdc54678b7cc49136f2d6af7|9ef432eb6251297304e76186b10a928d|delivered   |2017-10-02 10:56:33     |2017-10-02 11:07:15|2017-10-04 19:55:00         |2017-10-10 21:25:13          |2017-10-18 00:00:00          |
|53cdb2fc8bc7dce0b6741e2150273451|b0830fb4747a6c6d20dea0b8c802d7ef|delivered   |2018-07-24 20:41:37     |2018-07-26 03:24:27|2018-07-26 14:3

In [0]:
df_orders_selected =  (
    df_orders.select('order_id', 'customer_id', 'order_status', 
    expr('upper(order_status)'),
    expr('substring(order_approved_at, 0,4) as year'),
    expr('substring(order_approved_at, 6,2) as month'),
    expr('substring(order_approved_at, 9,2) as day'))   
)

In [0]:
df_orders_selected.selectExpr('order_id', 'customer_id','upper(order_status) as order_status','concat(year,"-",month,"-",day) as date').show(10)

+--------------------+--------------------+------------+----------+
|            order_id|         customer_id|order_status|      date|
+--------------------+--------------------+------------+----------+
|e481f51cbdc54678b...|9ef432eb625129730...|   DELIVERED|2017-10-02|
|53cdb2fc8bc7dce0b...|b0830fb4747a6c6d2...|   DELIVERED|2018-07-26|
|47770eb9100c2d0c4...|41ce2a54c0b03bf34...|   DELIVERED|2018-08-08|
|949d5b44dbf5de918...|f88197465ea7920ad...|   DELIVERED|2017-11-18|
|ad21c59c0840e6cb8...|8ab97904e6daea886...|   DELIVERED|2018-02-13|
|a4591c265e18cb1dc...|503740e9ca751ccdd...|   DELIVERED|2017-07-09|
|136cce7faa42fdb2c...|ed0271e0b7da060a3...|    INVOICED|2017-04-13|
|6514b8ad8028c9f2c...|9bdf08b4b3b52b552...|   DELIVERED|2017-05-16|
|76c6e866289321a7c...|f54a9f0e6b351c431...|   DELIVERED|2017-01-25|
|e69bfb5eb88e0ed6a...|31ad1d1b63eb99624...|   DELIVERED|2017-07-29|
+--------------------+--------------------+------------+----------+
only showing top 10 rows



###  Selecionando valores únicos

In [0]:
df_orders_selected.select('year').distinct().show()

+----+
|year|
+----+
|2016|
|2017|
|null|
|2018|
+----+



In [0]:
df_orders_selected.dropDuplicates(subset=['year']).show()

+--------------------+--------------------+------------+-------------------+----+-----+----+
|            order_id|         customer_id|order_status|upper(order_status)|year|month| day|
+--------------------+--------------------+------------+-------------------+----+-----+----+
|00b1cb0320190ca0d...|3532ba38a3fd24225...|    canceled|           CANCELED|null| null|null|
|d3c8851a6651eeff2...|957f8e082185574de...|  processing|         PROCESSING|2016|   10|  06|
|e481f51cbdc54678b...|9ef432eb625129730...|   delivered|          DELIVERED|2017|   10|  02|
|53cdb2fc8bc7dce0b...|b0830fb4747a6c6d2...|   delivered|          DELIVERED|2018|   07|  26|
+--------------------+--------------------+------------+-------------------+----+-----+----+



### Filtrando registros e condições

Operadores lógicos disponíveis:
* e: &
* ou: |
* não: ~

As funções `filter()` e `where()` podem ser utilizadas no processo de filtragem.

In [0]:
df_orders_selected.filter(~(col('year') == 'null')).show(10)

+--------------------+--------------------+------------+-------------------+----+-----+---+
|            order_id|         customer_id|order_status|upper(order_status)|year|month|day|
+--------------------+--------------------+------------+-------------------+----+-----+---+
|e481f51cbdc54678b...|9ef432eb625129730...|   delivered|          DELIVERED|2017|   10| 02|
|53cdb2fc8bc7dce0b...|b0830fb4747a6c6d2...|   delivered|          DELIVERED|2018|   07| 26|
|47770eb9100c2d0c4...|41ce2a54c0b03bf34...|   delivered|          DELIVERED|2018|   08| 08|
|949d5b44dbf5de918...|f88197465ea7920ad...|   delivered|          DELIVERED|2017|   11| 18|
|ad21c59c0840e6cb8...|8ab97904e6daea886...|   delivered|          DELIVERED|2018|   02| 13|
|a4591c265e18cb1dc...|503740e9ca751ccdd...|   delivered|          DELIVERED|2017|   07| 09|
|136cce7faa42fdb2c...|ed0271e0b7da060a3...|    invoiced|           INVOICED|2017|   04| 13|
|6514b8ad8028c9f2c...|9bdf08b4b3b52b552...|   delivered|          DELIVERED|2017

In [0]:
df_orders_selected.filter((col('year').isNull())).show(5)

+--------------------+--------------------+------------+-------------------+----+-----+----+
|            order_id|         customer_id|order_status|upper(order_status)|year|month| day|
+--------------------+--------------------+------------+-------------------+----+-----+----+
|00b1cb0320190ca0d...|3532ba38a3fd24225...|    canceled|           CANCELED|null| null|null|
|ed3efbd3a87bea76c...|191984a8ba4cbb214...|    canceled|           CANCELED|null| null|null|
|df8282afe61008dc2...|aa797b187b5466bc6...|    canceled|           CANCELED|null| null|null|
|8d4c637f1accf7a88...|b1dd715db389a2077...|    canceled|           CANCELED|null| null|null|
|7a9d4c7f9b0683378...|7f71ae48074c0cfec...|    canceled|           CANCELED|null| null|null|
+--------------------+--------------------+------------+-------------------+----+-----+----+
only showing top 5 rows



In [0]:
df_orders_selected.select('order_status').distinct().show()

+------------+
|order_status|
+------------+
|     shipped|
|    canceled|
|    invoiced|
|     created|
|   delivered|
| unavailable|
|  processing|
|    approved|
+------------+



In [0]:
(
    df_orders_selected.filter((col('year') == '2016') & (col('order_status') == 'invoiced'))
    .show(5)
)

+--------------------+--------------------+------------+-------------------+----+-----+---+
|            order_id|         customer_id|order_status|upper(order_status)|year|month|day|
+--------------------+--------------------+------------+-------------------+----+-----+---+
|c4e980a1d822db426...|88dc22aad9cf20898...|    invoiced|           INVOICED|2016|   10| 09|
|35b8e54d765e6b217...|1aaa5eaa9dd9bafb3...|    invoiced|           INVOICED|2016|   10| 07|
|711b9be9c346d9ecd...|81e4aed5ab4253757...|    invoiced|           INVOICED|2016|   10| 04|
|a6475bb7a50387e3c...|442d66f0d96f65609...|    invoiced|           INVOICED|2016|   10| 05|
|dd845e1cdb19f08d0...|01f7b7a4e25cda9ce...|    invoiced|           INVOICED|2016|   10| 10|
+--------------------+--------------------+------------+-------------------+----+-----+---+
only showing top 5 rows



In [0]:
(
    df_orders_selected.filter(((col('order_status') == 'unavailable') | (col('order_status') == 'canceled')) & (col('year') == '2017')).show(5)
  
)

+--------------------+--------------------+------------+-------------------+----+-----+---+
|            order_id|         customer_id|order_status|upper(order_status)|year|month|day|
+--------------------+--------------------+------------+-------------------+----+-----+---+
|8e24261a7e58791d1...|64a254d30eed42cd0...| unavailable|        UNAVAILABLE|2017|   11| 16|
|37553832a3a89c9b2...|7607cd563696c27ed...| unavailable|        UNAVAILABLE|2017|   08| 17|
|2f634e2cebf8c0283...|7353b0fb8e8d9675e...| unavailable|        UNAVAILABLE|2017|   09| 28|
|ee0db22a8e742b752...|aae50600d30bf2efe...| unavailable|        UNAVAILABLE|2017|   08| 24|
|6ad57aecbae806a7e...|d31dbd02ac052d662...| unavailable|        UNAVAILABLE|2017|   11| 30|
+--------------------+--------------------+------------+-------------------+----+-----+---+
only showing top 5 rows



In [0]:
(
    df_orders_selected.filter((col('order_status').isin('unavailable', 'canceled')) & (col('year') == '2017')).show(5)
)

+--------------------+--------------------+------------+-------------------+----+-----+---+
|            order_id|         customer_id|order_status|upper(order_status)|year|month|day|
+--------------------+--------------------+------------+-------------------+----+-----+---+
|8e24261a7e58791d1...|64a254d30eed42cd0...| unavailable|        UNAVAILABLE|2017|   11| 16|
|37553832a3a89c9b2...|7607cd563696c27ed...| unavailable|        UNAVAILABLE|2017|   08| 17|
|2f634e2cebf8c0283...|7353b0fb8e8d9675e...| unavailable|        UNAVAILABLE|2017|   09| 28|
|ee0db22a8e742b752...|aae50600d30bf2efe...| unavailable|        UNAVAILABLE|2017|   08| 24|
|6ad57aecbae806a7e...|d31dbd02ac052d662...| unavailable|        UNAVAILABLE|2017|   11| 30|
+--------------------+--------------------+------------+-------------------+----+-----+---+
only showing top 5 rows



In [0]:

(
    df_orders_selected
    .filter((col('order_status').isin('unavailable', 'canceled')))
    .filter((col('year') == '2017'))
    .show(5)
)

+--------------------+--------------------+------------+-------------------+----+-----+---+
|            order_id|         customer_id|order_status|upper(order_status)|year|month|day|
+--------------------+--------------------+------------+-------------------+----+-----+---+
|8e24261a7e58791d1...|64a254d30eed42cd0...| unavailable|        UNAVAILABLE|2017|   11| 16|
|37553832a3a89c9b2...|7607cd563696c27ed...| unavailable|        UNAVAILABLE|2017|   08| 17|
|2f634e2cebf8c0283...|7353b0fb8e8d9675e...| unavailable|        UNAVAILABLE|2017|   09| 28|
|ee0db22a8e742b752...|aae50600d30bf2efe...| unavailable|        UNAVAILABLE|2017|   08| 24|
|6ad57aecbae806a7e...|d31dbd02ac052d662...| unavailable|        UNAVAILABLE|2017|   11| 30|
+--------------------+--------------------+------------+-------------------+----+-----+---+
only showing top 5 rows



### Utilizando expressões no filtro

In [0]:
(
    df_orders_selected
    .filter('order_status in ("unavailable", "canceled") and year == "2017"')
    .show(5)
)

+--------------------+--------------------+------------+-------------------+----+-----+---+
|            order_id|         customer_id|order_status|upper(order_status)|year|month|day|
+--------------------+--------------------+------------+-------------------+----+-----+---+
|8e24261a7e58791d1...|64a254d30eed42cd0...| unavailable|        UNAVAILABLE|2017|   11| 16|
|37553832a3a89c9b2...|7607cd563696c27ed...| unavailable|        UNAVAILABLE|2017|   08| 17|
|2f634e2cebf8c0283...|7353b0fb8e8d9675e...| unavailable|        UNAVAILABLE|2017|   09| 28|
|ee0db22a8e742b752...|aae50600d30bf2efe...| unavailable|        UNAVAILABLE|2017|   08| 24|
|6ad57aecbae806a7e...|d31dbd02ac052d662...| unavailable|        UNAVAILABLE|2017|   11| 30|
+--------------------+--------------------+------------+-------------------+----+-----+---+
only showing top 5 rows



#### Observações
Quando nos referimos às colunas por meio da função `col()`, temos acesso à diversos métodos das colunas que podem ser utilizados para auxliar na filtragem do DataFrame. Alguns deles são:
* `isin()`: checa se a coluna contém os valores listados na função.
* `contains()`: utilizado para verificar se uma coluna de texto contém algum padrão especificado (não aceita regex). Aceita uma outra coluna de texto.
* `like()`: utilizado para verificar se uma coluna de texto contém algum padrão especificado (não aceita regex). Funciona de forma similar ao "LIKE" do SQL.
* `rlike()`: utilizado para verificar se uma coluna de texto contém algum padrão especificado (**aceita regex**). Funciona de forma similar ao "RLIKE" do SQL.
* `startswith()`: utilizado para verificar se uma coluna de texto começa com algum padrão especificado (**aceita regex**).
* `endswith()`: utilizado para verificar se uma coluna de texto termina com algum padrão especificado (**aceita regex**).
* `between()`: checa se os valores da coluna estão dentro do intervalo especificado. Os dois lados do intervalo são inclusivos.
* `isNull()`: retorna True se o valor da coluna é nulo
* `isNotNull()`: retorna True se o valor da coluna não é nulo

Outros métodos úteis:
* `alias()/name()`: usado para renomear as colunas em operações como select() e agg()
* `astype()/cast()`: usado para mudar o tipo das colunas. Aceita tanto um string como um tipo especificado pelo módulo pyspark.sql.types
* `substr()`: utilizado para cortar um string com base em índices dos caracteres 